Import Package

In [1]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib
import pandas_datareader as pdr
import mpl_finance as mpf
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

Selecting previously unselected package libta-lib0.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 3.9MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437792 sha256=9797127dd2b2e2ffa1e2837ecfca897b33af05e1c20344f19046ac68cb70e5eb
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib
Mounted at /

In [2]:
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  print(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int( classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2013/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [5]:
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", "2227", "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# stockNum = ["2330"]

In [ ]:
for days in list([5,10,15,20]):
  for stockno in stockNum:
    # load 進 五大價格 一個量
    new_df = pdr.DataReader(str(stockno)+'.TW', 'yahoo', start=start, end=end)
    new_df.index = new_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    new_df['Close'] = exponential_smoothing(days, 0.2, list(new_df['Close']))    
    #store file
    new_df = new_df[days:]
    new_df = new_df.reset_index()
    from google.colab import files
    new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/'+str(days)+'_smooth/'+str(stockno)+'.csv')
    # files.download('/content/drive/My Drive/Program_Trading_Code/'+str(days)+'_smooth/'+str(stockno)+'.csv')
    print(str(stockno)+" complete....")
  print(str(days)+'complete...................................=========================================')

In [ ]:
# days = 10
for days in list([5,10,15,20]):
  for pred_days in list([5,7,10,13,15]):
    wala_wala = 0
    wala_wala_our = 0
    wala_wala_origin = 0
    for stock_num in stockNum:
      new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/'+str(days)+'_smooth/'+str(stockno)+'.csv')
      #先算出之後回測用的價位
      Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
      Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
      Price_df = Price_df.reset_index()
      Price_df = Price_df[days:]
      Price_df = Price_df.reset_index()

      # set y label
      ans = []
      for i in range(0,len(new_df)-pred_days):
        Pi = new_df['Close'][i+pred_days]
        Pj = new_df['Close'][i]
        if Pi > Pj : y = 1
        else: y = 0
        ans.append(y)
      ans += [0]*pred_days
      new_df['y_label']= ans
      # start evaluaion ###############一般回測################################
      left_money = 10**6
      own_asset = 10**6
      time_slide = 30 
      own_stock = 0 
      money_record = [left_money]*time_slide
      origin_money = 10**6 #紀錄開場進 最後出
      origin_hold_stock = 0
      # print(new_df.shape)
      # print(Price_df.shape) 
      for i in range(time_slide, len(new_df)): 
        predict = int(new_df['y_label'][i])
        price = int(Price_df['Close'][i]*100)/100
        if i == time_slide:
          Affordstock = int(left_money /(price*1.00585))
          origin_hold_stock += Affordstock
          origin_money = origin_money - origin_hold_stock*price*1.00585
        elif i== len(new_df)-1:
          origin_money = origin_money +price*origin_hold_stock
        if predict == 1:
          if own_stock == 0:
            Afford_num = int(left_money /(price*1.00585))
            own_stock += Afford_num
            left_money = left_money - Afford_num*price*1.00585
            if str(stock_num) =="2327":
              print("買入")
              print("目前價格： " , price)
              print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
        elif predict == 0 :
          if own_stock > 0:
            left_money += own_stock*price
            own_stock = 0
            if str(stock_num) =="2327":
              print("賣出")
              print("目前價格： " , price)
              print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
        money_record.append(left_money+own_stock*price)
      print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
      print(str(stock_num)+"最後資產 ：", int(origin_money),"報酬率：",(int(origin_money)-(10**6))*100/(10**6),"%（開場進 最後出）")
      wala_wala += 10**6
      wala_wala_our += int(money_record[-1])
      wala_wala_origin += int(origin_money)
    print('n=' + str(days) + ', n2=' + str(pred_days) + " 測試結束")
    print("our總投資報酬率：", (wala_wala_our - wala_wala)*100 / wala_wala, "%")
    print("origin總投資報酬率：", (wala_wala_origin - wala_wala)*100 / wala_wala, "%")
    wala_wala = 0
    wala_wala_our = 0
    wala_wala_origin = 0

In [25]:
# for time in list([10,30]):
#  for pred_days in list([3,5,7,10,13,15]):
time =30
for out  in list([0.02,0.03,0.04,0.05,0.06,0.07]):
  for pred_days in list([5]):
    wala_wala = 0
    wala_wala_our = 0
    wala_wala_origin = 0
    for stock_num in stockNum:
      # print(stock_num)
      pd.set_option('display.max_columns', None)
      pd.set_option('display.max_rows', None)
      pd.set_option('max_colwidth',100)
      #start = '2013/01/01'
      start = '2019/05/13'
      end = '2020/09/21'
      s = start.split('/')
      e = end.split('/')
      start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
      end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))
      #=======================================================================================================
      df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
      df.index = df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
      df = df.reset_index()
      before = start - datetime.timedelta(days=120)
      df_big = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=before, end=end)
      df['MA30'] = talib.MA(np.array(df_big['Close']), timeperiod=time, matype=0)[-1*len(df):]
      #先算出之後回測用的價位
      Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
      Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
      Price_df = Price_df.reset_index()
      # Price_df = Price_df[days:]
      # Price_df = Price_df.reset_index()
      new_df = df
      # set y label
      ans = []
      for i in range(0,len(new_df)-pred_days):
        Pi = new_df['MA30'][i+pred_days]
        Pj = new_df['MA30'][i]
        if Pi > Pj : y = 1
        else: y = 0
        ans.append(y)
      ans += [0]*pred_days
      new_df['y_label']= ans

      # start evaluaion ###############一般回測################################
      left_money = 10**6
      own_asset = 10**6
      time_slide = 30 
      own_stock = 0 
      money_record = [left_money]*time_slide
      origin_money = 10**6 #紀錄開場進 最後出
      origin_hold_stock = 0
      # print(new_df.shape)
      # print(Price_df.shape) 
      # print("開始回測")
      y_preds_list = list(new_df['y_label'])
      # print(len(y_preds_list))
      stock_amount = [0]*(time)
      for i in range(time_slide, len(new_df)): 
        price = int(new_df['Close'][i]*100)/100
        for j in range(i-4, i):
          if y_preds_list[j] == 1 and stock_amount[j] != 0 and price > (int(new_df['Close'][j]*100)/100)*(1+out) : #如果今日價格小於他入場日的價格，就馬上出場 #更動：12月20
              own_stock -= stock_amount[j] #更新我本支股票累計的股數
              left_money += stock_amount[j]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
              stock_amount[j] = 0 #更新j天的股票數量為0
              # print("賺到提早出場", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
          #更動12月20
          if y_preds_list[j] == 1 and stock_amount[j] != 0 and price <(int(new_df['Close'][j]*100)/100)*(1+out) : #如果今日價格小於他入場日的價格，就馬上出場 #更動：12月20
            addition_money = min( ((left_money+own_stock*price)/5) , left_money)
            addition_money = 0 if addition_money < 0.0 else addition_money
            own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
            left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
            stock_amount[j]+= int( addition_money / ( price * (1 + 0.001425)) ) #
            # print("虧錢加碼", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)

        #預測5天後漲則今天進場，五天前如果有買則今天離場（30進35出）
        if y_preds_list[i] == 1: #如果我現在要進場買入
          #if y_preds_list[i-5] == 1: 
          if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
            # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
            addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
          else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
            # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
            addition_money = min( (left_money+own_stock*price)/5, left_money)
          addition_money = 0 if addition_money < 0.0 else addition_money
          own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
          left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
          stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
          # print("買", "股票進場數: ",stock_amount[-1], "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"這支股票的總資產：",left_money+own_stock*price)
        else: #如果我現在沒有要進場
          #if y_preds_list[i-5] == 1: 
          if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
            own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
            left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
            # print("賣", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
          #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
          stock_amount.append(0) #更新我本支股票當天的股數
        money_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
      # print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
      #這裡是教授策略＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠＠
      #   if y_preds_list[i] == 1: #如果我現在要進場買入
      #     if y_preds_list[i-5] == 1: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
      #       # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
      #       addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      #     else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
      #       # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
      #       addition_money = min( (left_money+own_stock*price)/5, left_money)
      #     addition_money = 0 if addition_money < 0.0 else addition_money
      #     own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      #     left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      #     stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      #   #print("買","股票進場數: ",stock_amount[-1],"現金: ",left_money ,"資產：",left_money+own_stock*price)
      #   else: #如果我現在沒有要進場
      #     if y_preds_list[i-5] == 1: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
      #       own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
      #       left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
      #       #print("賣","現金: ",left_money ,"資產：",left_money+own_stock*price)
      #     #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      #     stock_amount.append(0) #更新我本支股票當天的股數
      #   money_record.append(left_money+own_stock*price)
      # print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
      #這裡底下是原先在做n1n2
        # if i == time_slide:
        #   Affordstock = int(left_money / (price*1.00585))
        #   origin_hold_stock += Affordstock
        #   origin_money = origin_money - origin_hold_stock*price*1.00585
        # elif i == len(new_df)-1:
        #   origin_money = origin_money +price*origin_hold_stock
        # if predict == 1:
        #   if own_stock == 0:
        #     Afford_num = int(left_money /(price*1.00585))
        #     own_stock += Afford_num
        #     left_money = left_money - Afford_num*price*1.00585
        #     # print("買入")
        #     # print("目前價格： " , price, "日期: ",time_list[i])
        #     print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
        # elif predict == 0 :
        #   if own_stock > 0:
        #     left_money += own_stock*price
        #     own_stock = 0
        #     # print("賣出")
        #     # print("目前價格： " , price, "日期: ", time_list[i])
        #     print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
      # print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(10**6))*100/(10**6),"%")
      # print(str(stock_num)+"最後資產 ：", int(origin_money),"報酬率：",(int(origin_money)-(10**6))*100/(10**6),"%（開場進 最後出）")
      wala_wala += 10**6
      wala_wala_our += int(money_record[-1])
      wala_wala_origin += int(origin_money)
    print('out=' + str(out) + " 測試結束")
    print("our總投資報酬率：", (wala_wala_our - wala_wala)*100 / wala_wala, "%")
    print("origin總投資報酬率：", (wala_wala_origin - wala_wala)*100 / wala_wala, "%")
    wala_wala = 0
    wala_wala_our = 0
    wala_wala_origin = 0

out=0.02 測試結束
our總投資報酬率： 11.37200576923077 %
origin總投資報酬率： 0.0 %
out=0.03 測試結束
our總投資報酬率： 14.752901923076923 %
origin總投資報酬率： 0.0 %
out=0.04 測試結束
our總投資報酬率： 17.209978846153845 %
origin總投資報酬率： 0.0 %
out=0.05 測試結束
our總投資報酬率： 20.163746153846155 %
origin總投資報酬率： 0.0 %
out=0.06 測試結束
our總投資報酬率： 21.734226923076925 %
origin總投資報酬率： 0.0 %
out=0.07 測試結束
our總投資報酬率： 23.152525 %
origin總投資報酬率： 0.0 %


In [ ]:
def idxmax(x):
  max_num = max(x)
  idx = []
  for i in range(len(x)):
    if x[i] == max_num: idx.append(i)
  return idx

lst = [11,2,3,11,5,7,11]
print(idxmax(lst))

[0, 3, 6]


In [ ]:
def mode(x):
  import collections
  # calculate the frequency of each item
  data = collections.Counter(x)
  data_list = dict(data)
  # Print the items with frequency
  print(data_list)
  # Find the highest frequency
  max_value = max(list(data.values()))
  mode_val = [num for num, freq in data_list.items() if freq == max_value]
  return mode_val

print(mode([2,2,3,3,5,7,11]))

{2: 2, 3: 2, 5: 1, 7: 1, 11: 1}
[2, 3]
